In [74]:
#!/usr/bin/env python
# # -*- coding: utf-8 -*-

""" This module is meant to be used to explore a simplified version of the
    famous Turtle trading strategy.  Using cryptocurrency financial data from a csv
    file aggregated at Kaggle.com and given the name of a coin, time period of interest,
    and direction of strategy, these functions will create a relevant dataframe and
    price range, then find valid entry points, and finally return a list of price
    changes on all valid entry points selected.  This list can then be used for analysis
    to determine if the strategy shows promise to create positive expectancy.
"""

import pandas as pd
import numpy as np
from pandas.tseries.offsets import *

def create_coin_df(df, coin_name):
    ''' This function takes in a dataframe (pandas DataFrame) and a coin_name (string)
        and returns a new dataframe (pandas DataFrame) with Date as index,
        the column for name of coin, and the following price data columns, Open, High, Low,
        Close.
    '''
    # Print message if coin does not exist in csv file
    if ~df.Currency.str.contains(coin_name).any():
        print('Coin does not exist in database.  Will return empty dataframe.')
        
    # Create a dataframe of only specific coin and columns of interest
    df_coin = df[df.Currency == coin_name]
    df_coin = df_coin[['Currency', 'Date', 'Open','High', 'Low', 'Close']]

    # Set Date to a datetime object and use as index
    df_coin.Date = pd.to_datetime(df_coin.Date)
    df_coin = df_coin.set_index('Date')

    # Cast price data columns to floats
    df_coin.Open = df_coin.Open.apply(pd.to_numeric)
    df_coin.High = df_coin.High.apply(pd.to_numeric)
    df_coin.Low = df_coin.Low.apply(pd.to_numeric)
    df_coin.Close = df_coin.Close.apply(pd.to_numeric)

    # Sort data in ascending order
    df_coin.sort_index(axis=0, ascending=True, inplace=True)

    return df_coin


In [124]:
def create_range_column(df, period):
    ''' This function takes in a dataframe of price data and a integer for the length
        of the period desired for the range.  It then creates two new columns, one
        for the High desired period and one for the Low.  No return, changes input df.
    '''
    # Create new columns
    try:
        df['Range_High'] = df['High'].rolling(window=period).max().shift(1)
        df['Range_Low'] = df['Low'].rolling(window=period).min().shift(1)
    except ValueError:
        print('Period must be an integer value')
        

In [152]:
def generate_tradelist(df, period, direction='Both'):
    ''' Inputs are dataframe, period and flag for Long/Short/Both directions, function
        returns a list of price changes for all valid trades in the df given the parameters.
    '''
    # Create new df of potential trades, allowing for period number of days forward data
    df_trades = df[:-(period)]

    # Get a series of entry prices
    entry_prices_long = df_trades[df_trades.High > df_trades.Range_High].High + 1
    entry_prices_short = df_trades[df_trades.Low < df_trades.Range_Low].Low - 1

    # Create a series of exit prices based off closing price period days later
    exit_dates_long = entry_prices_long.index + DateOffset(days=period)
    exit_dates_short = entry_prices_short.index + DateOffset(days=period)

    exit_prices_long = []
    exit_prices_short = []

    for i in range(len(exit_dates_long)):
        exit_prices_long.append(df.loc[exit_dates_long[i]].Close)
    for j in range(len(exit_dates_short)):
        exit_prices_short.append(df.loc[exit_dates_short[j]].Close)

    # Generate the final list of price differences
    exit_prices_long_arr = np.array(exit_prices_long)
    exit_prices_short_arr = np.array(exit_prices_short)

    price_diffs_long = exit_prices_long_arr - entry_prices_long
    price_diffs_short = entry_prices_short - exit_prices_short_arr

    # Create list dependant on direction param
    if direction == 'Long':
        price_diffs_final = price_diffs_long
    elif direction == 'Short':
        price_diffs_final = price_diffs_short
    else:
        price_diffs_final = price_diffs_long.append(price_diffs_short)

    return price_diffs_final

In [153]:
# Read in all data from the Coinmarketcap file
df = pd.read_csv('consolidated_coin_data.csv', skiprows=4, low_memory=False)

In [166]:
# Get user input for coin name, period and direction
coin_name = input('What cryptocurrency do you want data for?\n')
period = int(input('How many days do you want to include in your break-out range?\n'))
direction = input('What trade direction do you want to include?\nLong/Short/Both\n')

What cryptocurrency do you want data for?
ethereum
How many days do you want to include in your break-out range?
100
What trade direction do you want to include?
Long/Short/Both
Both


In [167]:
print(coin_name, period, direction)

ethereum 100 Both


In [168]:
df_coin = create_coin_df(df, coin_name)
df_coin.head()

,Currency,Open,High,Low,Close
Date,,,,,
2015-08-07,ethereum,2.830000,3.540000,2.520000,2.770000
2015-08-08,ethereum,2.790000,2.800000,0.714725,0.753325
2015-08-09,ethereum,0.706136,0.879810,0.629191,0.701897
2015-08-10,ethereum,0.713989,0.729854,0.636546,0.708448
2015-08-11,ethereum,0.708087,1.130000,0.663235,1.070000


In [169]:
create_range_column(df_coin, period)
df_coin.tail()

,Currency,Open,High,Low,Close,Range_High,Range_Low
Date,,,,,,,
2018-02-07,ethereum,790.43,851.20,723.43,757.07,1432.88,281.17
2018-02-08,ethereum,755.84,845.09,755.84,817.81,1432.88,281.17
2018-02-09,ethereum,818.48,884.00,788.83,883.87,1432.88,281.17
2018-02-10,ethereum,882.47,917.40,825.58,860.41,1432.88,281.17
2018-02-11,ethereum,859.29,859.29,788.22,814.66,1432.88,287.69


In [170]:
price_diff_list = generate_tradelist(df_coin, period, direction)

In [171]:
print(len(price_diff_list), price_diff_list.mean())

67 89.7379104477612


In [172]:
price_diff_list

Date
2016-01-18      5.22
2016-01-20      4.84
2016-01-23      7.13
2016-01-24      6.01
2016-01-25      5.75
2016-01-26      5.99
2016-02-07      8.14
2016-02-08      9.29
2016-02-09      9.73
2016-02-10      7.78
2016-02-11      6.73
2016-02-12      6.71
2016-02-25      6.09
2016-03-01      5.49
2016-03-02      4.16
2016-03-03      3.09
2016-03-04      4.33
2016-03-05      4.33
2016-03-12     -2.69
2016-03-13     -2.95
2016-06-12     -2.47
2016-06-13     -4.87
2016-06-14     -7.02
2016-06-16     -8.95
2016-06-17     -9.42
2017-02-14    160.12
2017-02-25    233.74
2017-02-26    248.93
2017-02-27    241.39
2017-02-28    244.61
               ...  
2017-03-16    275.53
2017-03-17    247.14
2017-04-27    193.34
2017-04-28    189.72
2017-04-29    196.28
2017-04-30    216.75
2017-05-01    212.51
2017-05-04    212.04
2017-05-05    193.83
2017-05-19    213.33
2017-05-20    213.11
2017-05-21    210.64
2017-05-22    175.74
2017-05-24    158.37
2017-05-30     94.72
2017-05-31     58.53
2017-06-

In [ ]:
# Run through the functions to generate the correct trade data
df_coin = create_coin_df(df, coin_name)

print('These are the first 10 rows of the new {} dataframe.'.format(coin_name))
print(df_coin.head(10))

create_range_column(df_coin, period)

print('Added new columns to the dataframe')
print(df_coin.head())

price_diff_list = generate_tradelist(df_coin, period, direction)
trade_count = len(price_diff_list)
ave_price_diff = price_diff_list.mean()

print('Total number of trades: {}'.format(trade_count))
print('Average price change per trade: {}'.format(ave_price_diff))

In [45]:
create_range_column(df, 'test')
df.head()

Period must be an integer value


,Currency,Date,Open,High,Low,Close,Volume,Market Cap
0,0x,"Feb 11, 2018",1.09,1.09,0.934155,0.979054,"4,888,770","555,363,000"
1,0x,"Feb 10, 2018",1.14,1.20,0.985705,1.10,"10,828,700","576,535,000"
2,0x,"Feb 09, 2018",1.08,1.15,1.01,1.14,"5,979,420","545,842,000"
3,0x,"Feb 08, 2018",0.988963,1.13,0.988963,1.07,"12,992,800","501,142,000"
4,0x,"Feb 07, 2018",1.01,1.16,0.902145,0.993355,"13,476,600","513,163,000"
